In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import math
import warnings
from matplotlib import rcParams

import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow import keras
from tensorflow.python.keras.layers import Dropout, Dense, LSTM,Activation,GRU,SimpleRNN
from tensorflow.python.keras import backend as K

from keras import regularizers
from keras.losses import mean_squared_error

warnings.filterwarnings("ignore")

data_init = pd.read_excel("shujuji.xlsx")
x,y = data_init.iloc[:,1:6],data_init.iloc[:,-1]
data_init.iloc[:,1:].agg(['min','max', "mean"])
data_init.iloc[:,1:-1]

scaler_input = MinMaxScaler(feature_range=(0, 1))

scaler_output = MinMaxScaler(feature_range=(0, 1))

data_input = data_init.iloc[:,1:-1]
data_output = np.array(data_init.iloc[:,-1]).reshape(-1, 1)


x_data = scaler_input.fit_transform(data_input)
y_data = scaler_output.fit_transform(data_output)

time_steps = 2

xx_data = []
yy_data = []

for i in range(time_steps, len(x_data)):
    xx_data.append(x_data[i - time_steps:i, :].tolist())
    yy_data.append(y_data[i])
    
yy_data = np.array(yy_data)

np.array(xx_data).shape,np.array(yy_data).shape

train_X_nn, train_y_nn = np.array(xx_data)[:184,:], yy_data[:184]
test_X_nn, test_y_nn = np.array(xx_data)[184:,:], yy_data[184:]

x_train,y_train = np.array(train_X_nn),np.array(train_y_nn)

x_test,y_test = np.array(test_X_nn),np.array(test_y_nn)


def show_data(test_y,predict_y,title):
    plt.figure(figsize=(10,8),dpi=100)
    x = [i for i in range(test_y.shape[0])]
    plt.plot(x, test_y, color='navy',lw = 2, label='Actual data')
    plt.plot(x, predict_y, color='c',lw = 2, label='Predict data')
    x_label = [i for i in x[::5]]
    
    y_label = [j for j in range(0,500,50)]
    plt.yticks(y_label)
    plt.xticks(x_label)
    plt.xlabel('Time/d')
    plt.ylabel('COD')
    plt.title(title)
    plt.legend()
    plt.show()

In [21]:
from torch import nn
import torch
import math as mt

class PositionalEncoding(nn.Module):
    
    def __init__(self, d_model, max_len=5000):
        
        super(PositionalEncoding, self).__init__()      
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-mt.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]
          

class TransAm(nn.Module):
    def __init__(self,feature_size=250,num_layers=1,dropout=0.1):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'
        
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=2, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        self.decoder = nn.Linear(2*feature_size,1)
        self.init_weights()
        
        self.feature_size = feature_size
        self.num_layers   = num_layers
        self.dropout      = dropout
        
    def feature(self):
        return{"feature_size":self.feature_size,"num_layers":self.num_layers,"dropout":self.dropout}
        

    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self,src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src,self.src_mask)
        output = output.view(output.shape[0], -1)
        output = self.decoder(output)
        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [22]:
import os
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt

import time
from sklearn import metrics

import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from torch.autograd import Variable
import numpy as np

import math


class General_Regression_Training_3d():

    def fitness(self, evaluationStr="r2"):
        if (evaluationStr == "r2"):
            return self.r2
        elif (evaluationStr == "r2_adjusted"):
            return self.r2_adjusted
        elif (evaluationStr == "rmsle"):
            return self.rmsle
        elif (evaluationStr == "mape"):
            return self.mape
        elif (evaluationStr == "r2_adjusted"):
            return self.r2_adjusted
        elif (evaluationStr == "mad"):
            return self.mad
        elif (evaluationStr == "mae"):
            return self.mae
        
    def save_results(self):
        resultTitle     = [str(line) for line in self.resultDict.keys()]
        resultList      = [ "_".join([ str(l) for l in line]) if isinstance(line,list) else str(line) for line in self.resultDict.values()]
        y_test          = self.y_test
        test_prediction = self.test_prediction
        save_path       = self.save_path

        save_result = "/".join([save_path, 'result.csv'])
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        
        try:
            count = len(open(save_result, 'rU').readlines())
        except:
            count = 1

        resultTitle.insert(0, "count")
        resultList.insert(0, str(count))
        
        if not os.path.exists(save_result):
            with open(save_result, 'w') as f:
                titleStr = ",".join(resultTitle)
                f.write(titleStr)
                f.write('\n')
        
        with open(save_result, 'a+') as f:
            contentStr = ",".join(resultList)
            f.write(contentStr)
            f.write('\n')
        Loss_path = os.path.join(save_path, 'Loss')
        if not os.path.exists(Loss_path):
            os.makedirs(Loss_path)
        
        save_Loss = os.path.join(Loss_path, str(count) + '.csv')
        
        df = pd.DataFrame()
        df["TrainLoss"] = self.TrainLosses
        df["TestLoss"] = self.TestLosses
        df.to_csv(save_Loss, index=False)
        pred_path = os.path.join(save_path, 'Prediction')
        if not os.path.exists(pred_path):
            os.makedirs(pred_path)

        save_prediction = os.path.join(pred_path, str(count) + '.csv')
        df = pd.DataFrame()

        df["y_test"] = [i for i in y_test]
        df["test_prediction"] =[i for i in test_prediction]
        df.to_csv(save_prediction, index=False)

        print('Save the value of prediction successfully!!')

        model_path = os.path.join(save_path, 'Model')
        if not os.path.exists(model_path):
            os.makedirs(model_path)

        if(self.use_more_gpu):
            torch.save(self.net.state_dict(), os.path.join(model_path, str(count) + ".pth"))
        else:
            torch.save(self.net.state_dict(), os.path.join(model_path, str(count) + ".pth"))

        return count


    def reg_calculate(self,true, prediction, features=None):
        '''
            To calculate the result of regression,
            including mse, rmse, mae, r2, four criterions.
        '''
        prediction[prediction < 0] = 0

        mse = metrics.mean_squared_error(true, prediction)
        rmse = np.sqrt(mse)

        mae = metrics.mean_absolute_error(true, prediction)
        mape = np.mean(np.abs((true - prediction) / true)) * 100

        r2 = metrics.r2_score(true, prediction)
        rmsle = np.sqrt(metrics.mean_squared_log_error(true, prediction))

        try:
            n = len(true)
            p = features
            r2_adjusted = 1-((1-metrics.r2_score(true, prediction))*(n-1))/(n-p-1)
        except:
            print('if you wanna get the value of r2_adjusted, you can define the number of features, '
                  'which is the third parameter.')
            return mse, rmse, mae, mape, r2, rmsle

        return mse, rmse, mae, mape, r2, r2_adjusted, rmsle


    def __init__(self,net,learning_rate = [1e-3,1e-5,1e-7], batch_size = 1024, epoch = 2000, use_more_gpu = False,weight_decay=1e-8, device=0 ,save_path='CNN_Result'):

        self.net = net
        self.resultDict = {"learning_rate":learning_rate,"batch_size":batch_size,"epoch":epoch,"weight_decay":weight_decay,"use_more_gpu":use_more_gpu,"device":device,}
        self.resultDict = dict(self.resultDict,**self.net.feature())

        self.batch_size = batch_size
        self.use_more_gpu = use_more_gpu
        self.lr = learning_rate
        self.epoch = epoch
        self.weight_decay = weight_decay
        self.device = device
        self.epoch = epoch
        
        self.save_path = save_path
        if not os.path.exists(self.save_path):
            os.makedirs(self.save_path)

        self.avgLossList = []
        self.TrainLosses = []
        self.TestLosses = []
        self.t = 0
        self.D = []
        self.n = 0
        self.limit = [1e-5, 1e-6, 1e-7]
        
    def create_batch_size(self, X_train, y_train):
        p = np.random.permutation(X_train.shape[0])
        data = X_train[p]
        label = y_train[p]

        batch_size = self.batch_size
        batch_len = X_train.shape[0] // batch_size + 1

        b_datas = []
        b_labels = []
        for i in range(batch_len):
            try:
                batch_data = data[batch_size * i: batch_size * (i + 1)]
                batch_label = label[batch_size * i: batch_size * (i + 1)]
            except:
                batch_data = data[batch_size * i: -1]
                batch_label = label[batch_size * i: -1]
            b_datas.append(batch_data)
            b_labels.append(batch_label)

        return b_datas, b_labels
    
    
        
    # 訓練函數
    def fit(self, X_train, y_train, X_test, y_test):
        ''' training the network '''
        if y_train.ndim == 1:
            y_train = y_train.reshape(-1, 1)
        
        if y_test.ndim == 1:
            y_test = y_test.reshape(-1, 1)
        
        self.X_train, self.X_test, self.y_train, self.y_test = X_train, X_test, y_train, y_test
        

        b_data, b_labels = self.create_batch_size(X_train, y_train)
        
        save_result = os.path.join(self.save_path, 'Results.csv')
        try:
            count = len(open(save_result, 'rU').readlines())
        except:
            count = 1

        net_weight = os.path.join(self.save_path, 'Weight')
        if not os.path.exists(net_weight):
            os.makedirs(net_weight)
        
        net_path = os.path.join(net_weight, str(count) + '.pkl')
        net_para_path = os.path.join(net_weight, str(count) + '_parameters.pkl')
        
        
        device = torch.device(self.device if torch.cuda.is_available() else "cpu")
        if torch.cuda.is_available():
            print("Let's use GPU: {}".format(self.device))
        else:
            print("Let's use CPU")
            
            
        
        if self.use_more_gpu and torch.cuda.device_count() > 1:
            print("Let's use", torch.cuda.device_count(), "GPUs")
            self.net = nn.DataParallel(self.net)
        self.net.to(device)
        
        self.net.train()
        try:
            optim = torch.optim.Adam(self.net.parameters(), lr=self.lr[0], weight_decay=self.weight_decay)
        except:
            optim = torch.optim.Adam(self.net.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        criterion = torch.nn.MSELoss()
        print("")

        start = time.time()
        limit = self.limit[0]
        for e in range(self.epoch):
            
            tempLoss = []
            # 訓練模式
            self.net.train()
            for i in range(len(b_data)):
                if torch.cuda.is_available():
                    train_x = Variable(torch.FloatTensor(b_data[i])).to(device)
                    train_y = Variable(torch.FloatTensor(b_labels[i])).to(device)
                else:
                    train_x = Variable(torch.FloatTensor(b_data[i]))
                    train_y = Variable(torch.FloatTensor(b_labels[i]))


                prediction = self.net(train_x)
                
                loss = criterion(prediction, train_y)
                tempLoss.append(float(loss))
                
                optim.zero_grad()
                loss.backward()
                optim.step()

            self.D.append(loss.cpu().data.numpy())
            avgloss =  np.array(tempLoss).sum() / len(tempLoss)
            self.avgLossList.append(avgloss)
            
            
            if( ( e + 1 ) % 100 == 0):
                print('Training... epoch: {}, loss: {}'.format((e + 1), self.avgLossList[-1]))

                self.net.eval()
                if torch.cuda.is_available():
                    test_x = Variable(torch.FloatTensor(self.X_test)).to(device)
                    test_y = Variable(torch.FloatTensor(self.y_test)).to(device)
                else:
                    test_x = Variable(torch.FloatTensor(self.X_test))
                    test_y = Variable(torch.FloatTensor(self.y_test))

                test_prediction = self.net(test_x)
                test_loss = criterion(test_prediction, test_y)

                self.TrainLosses.append(avgloss)
                self.TestLosses.append(test_loss.cpu().data.numpy())

                self.test_prediction = test_prediction.cpu().data.numpy()
                self.test_prediction[self.test_prediction < 0] = 0
                
                
            
            if len(self.D) >= 20:
                loss1 = np.mean(np.array(self.D[-20:-10]))
                loss2 = np.mean(np.array(self.D[-10:]))
                d = float(np.abs(loss2 - loss1))

                
                if d < limit or e == self.epoch-1  or e > (self.epoch-1)/3 * (self.n + 1): 
                    self.D = []
                    self.n += 1
                    print('The error changes within {}'.format(limit))
                    self.e = e + 1
                    print(
                        'Training... epoch: {}, loss: {}'.format((e + 1), loss.cpu().data.numpy()))
                    
                    
                    torch.save(self.net, net_path)
                    torch.save(self.net.state_dict(), net_para_path)
                    
                    self.net.eval()
                    if torch.cuda.is_available():
                        test_x = Variable(torch.FloatTensor(self.X_test)).to(device)
                        test_y = Variable(torch.FloatTensor(self.y_test)).to(device)
                    else:
                        test_x = Variable(torch.FloatTensor(self.X_test))
                        test_y = Variable(torch.FloatTensor(self.y_test))
                    
                    test_prediction = self.net(test_x)
                    test_loss = criterion(test_prediction, test_y)

                
                    self.test_prediction = test_prediction.cpu().data.numpy()
                    self.test_prediction[self.test_prediction < 0] = 0
                    
                    
                    self.mse, self.rmse, self.mae, self.mape, \
                        self.r2, self.r2_adjusted, self.rmsle = self.reg_calculate(self.y_test, self.test_prediction  ,self.X_test.shape[-1] )
                    print('\033[1;35m Testing... epoch: {}, loss: {} , r2 {}\033[0m!'.format((e + 1), test_loss.cpu().data.numpy(), self.r2))

                    if self.n == 3:
                        print('The meaning of the loop is not big, stop!!')
                        break
                    limit = self.limit[self.n]
                    print('Now learning rate is : {}'.format(self.lr[self.n]))
                    optim.param_groups[0]["lr"] = self.lr[self.n]
            
            
        end = time.time()
        self.t = end - start
        print('Training completed!!! Time consuming: {}'.format(str(self.t)))


        resDict = {"mse":self.mse, "rmse":self.rmse, "mae":self.mae, "mape":self.mape, "r2":self.r2, "r2_adjusted":self.r2_adjusted, "rmsle":self.rmsle}
        self.resultDict = dict(resDict,**self.resultDict)

        self.mse, self.rmse, self.mae, self.mape, \
        self.r2, self.r2_adjusted, self.rmsle = self.reg_calculate(self.y_test, self.test_prediction,
                                                                    self.X_test.shape[-1])
        
    
    def fitness(self, evaluationStr = "r2"):
        if(evaluationStr == "r2"):
            return self.r2
        elif(evaluationStr == "r2_adjusted"):
            return  self.r2_adjusted
        elif(evaluationStr == "rmsle"):
            return  self.rmsle
        elif(evaluationStr == "mape"):
            return  self.mape
        elif(evaluationStr == "r2_adjusted"):
            return  self.r2_adjusted
        elif(evaluationStr == "mad"):
            return  self.mad
        elif(evaluationStr == "mae"):
            return  self.mae
            
        
        

In [23]:
model = TransAm(feature_size=6,num_layers=1,dropout=0.5)

grt = General_Regression_Training_3d(model,learning_rate = [1e-3,1e-6,1e-8],batch_size = 512,use_more_gpu = False,weight_decay=1e-3, device=0 ,save_path='transformer_Result',epoch = 20000)

grt.fit(x_train, y_train, x_test, y_test)

grt.save_results()

Let's use CPU

Training... epoch: 100, loss: 0.07894935458898544
The error changes within 1e-05
Training... epoch: 118, loss: 0.08207841217517853
 Testing... epoch: 118, loss: 0.1043805330991745 , r2 -0.3325662569176355!
Now learning rate is : 1e-06
The error changes within 1e-06
Training... epoch: 152, loss: 0.08077182620763779
 Testing... epoch: 152, loss: 0.10437044501304626 , r2 -0.33243742724188485!
Now learning rate is : 1e-08
Training... epoch: 200, loss: 0.07946425676345825
Training... epoch: 300, loss: 0.08509024232625961
Training... epoch: 400, loss: 0.0850074291229248
Training... epoch: 500, loss: 0.07692050188779831
Training... epoch: 600, loss: 0.0816832184791565
Training... epoch: 700, loss: 0.08173184096813202
Training... epoch: 800, loss: 0.08006345480680466
Training... epoch: 900, loss: 0.07815998047590256
Training... epoch: 1000, loss: 0.07792369276285172
Training... epoch: 1100, loss: 0.07802042365074158
Training... epoch: 1200, loss: 0.08125874400138855
Training... 

1

In [28]:
transfosmer_df = pd.read_csv(r".\transformer_Result\Prediction\transformer.csv")
test_y = []
test_prediction=[]
for index in transfosmer_df["y_test"]:
    tmp = []
    tmp.append(float(index[1:-1]))
    test_y.append(tmp)

for index in transfosmer_df["test_prediction"]:
    tmp = []
    tmp.append(float(index[1:-1]))
    test_prediction.append(tmp)
test_y = scaler_output.inverse_transform(test_y)
test_prediction = scaler_output.inverse_transform(test_prediction)


data = {"y_test": [index[0] for index in test_y.tolist()], "y_predict_Transformer": [index[0] for index in test_prediction.tolist()]}
print(data)
frame = pd.DataFrame(data)
frame.to_csv("Transformer_data.csv", index = False)

{'y_test': [34.026099995904, 28.159100080512, 25.587200086656, 49.76740006752, 49.135399981952, 45.713300091712, 23.889299893824, 38.492600042048, 25.35079999872, 42.814899860864, 21.822799905215998, 24.146900016832, 43.969900098688, 26.67690009856, 21.17810009056, 45.795099878975996, 39.81759985376, 45.635300136704, 37.899299868928, 43.01880009152, 28.201399881024, 43.304800025728, 41.313800094656, 43.249100096384, 22.330700105471998, 23.3522001056, 27.319499904128, 32.305900074944, 27.322000099136, 22.681800023872, 21.49720001216, 38.124299865024, 35.883300070528, 44.661800131776, 33.227200066496, 35.120900072384, 39.325500005248, 40.38390012256, 31.690900006272, 30.325199947584004, 48.764000074879995, 29.737100108672, 34.22369990176, 25.987699907136, 47.930899869952, 39.931000046975996, 37.50219999232, 41.32560007487999, 20.2241, 29.650199959295996, 48.264000050496, 38.044700058944, 30.091899887232003, 25.423399972864, 22.113400043520002, 28.985700016447996, 34.845099969792, 43.6293